In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np
import pickle

In [2]:
model = load_model('model.h5')

In [3]:
with open('onehot_encoder_geo.pkl', 'rb') as file:
    onehot_encoder_geo = pickle.load(file)
    
with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [4]:
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age':	48,
    'Tenure': 7,
    'Balance': 140000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 65000
}

In [5]:
input_data

{'CreditScore': 600,
 'Geography': 'France',
 'Gender': 'Male',
 'Age': 48,
 'Tenure': 7,
 'Balance': 140000,
 'NumOfProducts': 2,
 'HasCrCard': 1,
 'IsActiveMember': 1,
 'EstimatedSalary': 65000}

In [6]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,48,7,140000,2,1,1,65000


In [7]:
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,48,7,140000,2,1,1,65000


In [8]:
geo_onehot = onehot_encoder_geo.transform(input_df[['Geography']])
geo_onehot.toarray()

array([[1., 0., 0.]])

In [9]:
onehot_encoder_geo.get_feature_names_out()

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [10]:
input_df = input_df.drop(input_df[['Geography']], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,1,48,7,140000,2,1,1,65000


In [11]:
geo_df = pd.DataFrame(geo_onehot.toarray(), columns=onehot_encoder_geo.get_feature_names_out())
geo_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [12]:
input_df = pd.concat([input_df, geo_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,48,7,140000,2,1,1,65000,1.0,0.0,0.0


In [13]:
# Scaling the input data
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.86537308,  0.69270405,  1.02306797,
         0.80843615,  0.64920267,  0.97481699, -0.61603215,  1.00150113,
        -0.57946723, -0.57638802]])

In [14]:
# Predict Churn
prediction = model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 126ms/step


array([[0.1763538]], dtype=float32)

In [15]:
prediction_proba = prediction[0][0]

In [17]:
prediction_proba

0.1763538

In [18]:
if prediction_proba > 0.5:
    print('the customer is likely to churn')
else:
    print('the customer is not likely to churn')

the customer is not likely to churn
